# Text Classification with Naive Bayes

### Importing, preparing and splitting data

In [1]:
# finding path of data
import os
print(os.getcwd())
print(os.listdir('.'))
print(os.listdir('C:/Users/Jannik/Coding Ninjas/Machine Learning/Naive Bayes/20_newsgroups.tar/20_newsgroups'))

C:\Users\Jannik\Coding Ninjas\Machine Learning\Naive Bayes
['.ipynb_checkpoints', '20_newsgroups.tar', 'Naive_Bayes.ipynb', 'Notes.ipynb', 'Text_Classification_Project.ipynb']
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [2]:
# creating access to all data in all files with dictionary
lst_names = []
d = {}

for (root,dirs,files) in os.walk('C:/Users/Jannik/Coding Ninjas/Machine Learning/Naive Bayes/20_newsgroups.tar/20_newsgroups'):
     for name in dirs:
        lst_names.append(name)
        
for key in lst_names:
    d[key] = []
    
for key in d.keys():
    for (root,dirs,files) in os.walk('C:/Users/Jannik/Coding Ninjas/Machine Learning/Naive Bayes/20_newsgroups.tar/20_newsgroups/{}'.format(key)):
         for name in files:
            d[key].append(name)


In [3]:
len(d)

20

In [4]:
# importing stopwords from nltk
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [5]:
# creating vocabulary for 80% of documents of every class
direction = 'C:/Users/Jannik/Coding Ninjas/Machine Learning/Naive Bayes/20_newsgroups.tar/20_newsgroups'
import re
def create_voc(dic,direction):
    voc = {} 
    for i in range(0,len(dic)):
        train_split = int(0.8 * len(list(dic.values())[i]))
        # just to see work progress
        print(str(i+1)+'/'+ str(len(dic)),end=' ')
        for j in range(0,train_split):
            data = open('{}/{}/{}'.format(direction,(list(dic.keys())[i]),(list(dic.values())[i][j])))
            data = data.read()
            data = data.lower()
            # extract words from string using regex
            split = re.findall(r'\w+', data)
            for word in split:
                if word not in stopwords:
                    if word not in voc:
                        voc[word] = 1
                    else:
                        voc[word] += 1
    # sorting voc
    sort_voc = sorted(voc.items(), key=lambda x: x[1], reverse=True)
    # just considering 1000 most important words
    new_voc = {}
    for i in range(1000):
        new_voc[sort_voc[i][0]] = sort_voc[i][1]
    
    # deleting unnecesary space use
    del(voc)
    del(sort_voc)
        
    return new_voc


In [6]:
# creating the vocabulary from our training data
#vocabulary = create_voc(d,direction)

In [7]:
#checking vocabulary
#print(vocabulary)
#len(vocabulary)

In [8]:
# creating 2d arrays for data using vocabulary and our dictionary/direction to access all the data
def create_array(voc,dic,direction,test=False):
    # creating x,y and feature arrays
    x = []
    y = []
    features = [key for key in voc.keys()]
    
    # set row count index independent from test or train split
    c = 0
    
    # if test is True, we are creating test arrays
    for i in range(0,len(dic)):
        if test == False:
            array_split = int(0.8 * len(list(dic.values())[i]))
            start = 0
        else:
            array_split = len(list(dic.values())[i])
            start = array_split - (array_split - (int(0.8 * array_split)))
            
        # just to see work progress
        print(str(i+1)+'/'+ str(len(dic)),end=' ')
        
        for j in range(start,array_split):
            y.append(list(dic.keys())[i])
            data = open('{}/{}/{}'.format(direction,(list(dic.keys())[i]),(list(dic.values())[i][j])))
            data = data.read()
            data = data.lower()
            # extract words from documents of each class using regex and put counts in x_array
            split = re.findall(r'\w+', data)
            x.append([0 for f in range(len(features))])
            for g in range(len(split)):
                if split[g] in features:
                    index = features.index(split[g])
                    x[c][index] += 1
            c += 1
    return x,y    

In [9]:
import numpy as np

In [10]:
#creating training arrays and converting them to numpy arrays
#x_train,y_train = create_array(vocabulary,d,direction,test=False)
#x_train = np.array(x_train)
#y_train = np.array(y_train)

In [11]:
#n = 13039
#x_train[n,:], y_train[n]

In [12]:
#creating testing arrays and converting them to numpy arrays
#x_test,y_test = create_array(vocabulary,d,direction,test=True)
#x_test = np.array(x_test)
#y_test = np.array(y_test)

In [13]:
#x_train.shape

In [14]:
# checking validity of arrays
#len(x_test),len(y_test)

In [15]:
# checking validity of arrays
#len(x_train),len(y_train)

### Naive Bayes Classifier

In [16]:
# fit function for classifier with training data
def fit(x_train,y_train):
    result = {}
    count_classes = set(y_train)
    for current_class in count_classes:
        result[current_class] = {}
        result['total_data'] = len(y_train)
        num_features = x_train.shape[1]
        curr_class_rows =  (y_train == current_class)
        x_train_current = x_train[curr_class_rows]
        y_train_current = y_train[curr_class_rows]
        result[current_class]['total_count_class'] = len(y_train_current)
        for j in range(1, num_features +1):
            result[current_class][j] = {}
            all_possible_values = set(x_train[:,j-1])
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (current_value == x_train_current[:,j-1]).sum()
                
    return result   

In [17]:
# finding probabilities of current class
def probabilities(x,dictionary,current_class):
    # total data belonging to current class devided with total data (2nd term of formula)
    output = np.log(dictionary[current_class]['total_count_class']) - np.log(dictionary['total_data'])
    num_features = len(dictionary[current_class].keys())-1
    for j in range(1,num_features + 1):
        xj = x[j-1]
        #print('xj',xj)
        # error exception
        if xj not in dictionary[current_class][j].keys():
            count_current_class_with_value_xj = 1
        else:    
            # with lapace correction added
            count_current_class_with_value_xj = dictionary[current_class][j][xj]  + 1 
        count_current_class = dictionary[current_class]['total_count_class'] + len(dictionary[current_class][j].keys())
        # 1st term of formula
        prob_xj = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
        output += prob_xj
    return output

In [18]:
# find prediction for single x points
def predict_single_point(dictionary,x):
    # initial values
    best_p = -1000
    best_class = -1
    classes = dictionary.keys()
    first_run = True
    for current_class in classes:
        #print('current_class:', current_class)
        # avoiding total data as a class
        if current_class == 'total_data':
            continue
        p_current_class = probabilities(x,dictionary,current_class)
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class

In [19]:
# predict function
def predict(dictionary,x_test):
    y_pred = []
    # progress count 
    c = 0
    print('predict progress: ')
    for x in x_test:
        c += 1
        # progress overview
        if c % 100 == 0:
            print(c, '/', len(x_test), end=' ')
        x_class = predict_single_point(dictionary,x)
        y_pred.append(x_class)
    return y_pred

## Main

In [20]:
#creating the vocabulary from our training data from the computer
print('vocabulary progress:')
vocabulary = create_voc(d,direction)
print()

#creating training arrays and converting them to numpy arrays
print('training arrays progress:')
x_train,y_train = create_array(vocabulary,d,direction,test=False)
x_train = np.array(x_train)
y_train = np.array(y_train)
print()

# fit function for classifier with training data
dictionary = fit(x_train,y_train)

# creating testing arrays and converting them to numpy arrays
print('testing array progress:')
x_test,y_test = create_array(vocabulary,d,direction,test=True)
x_test = np.array(x_test)
y_test = np.array(y_test)

vocabulary progress:
1/20 2/20 3/20 4/20 5/20 6/20 7/20 8/20 9/20 10/20 11/20 12/20 13/20 14/20 15/20 16/20 17/20 18/20 19/20 20/20 
training arrays progress:
1/20 2/20 3/20 4/20 5/20 6/20 7/20 8/20 9/20 10/20 11/20 12/20 13/20 14/20 15/20 16/20 17/20 18/20 19/20 20/20 
testing array progress:
1/20 2/20 3/20 4/20 5/20 6/20 7/20 8/20 9/20 10/20 11/20 12/20 13/20 14/20 15/20 16/20 17/20 18/20 19/20 20/20 

In [21]:
# prediction y_pred for x_test data
y_pred = predict(dictionary,x_test)
#print(set(y_pred))

predict progress: 
100 / 4000 200 / 4000 300 / 4000 400 / 4000 500 / 4000 600 / 4000 700 / 4000 800 / 4000 900 / 4000 1000 / 4000 1100 / 4000 1200 / 4000 1300 / 4000 1400 / 4000 1500 / 4000 1600 / 4000 1700 / 4000 1800 / 4000 1900 / 4000 2000 / 4000 2100 / 4000 2200 / 4000 2300 / 4000 2400 / 4000 2500 / 4000 2600 / 4000 2700 / 4000 2800 / 4000 2900 / 4000 3000 / 4000 3100 / 4000 3200 / 4000 3300 / 4000 3400 / 4000 3500 / 4000 3600 / 4000 3700 / 4000 3800 / 4000 3900 / 4000 4000 / 4000 

### sklearn build in function

In [22]:
# Gaussian Classifier
from sklearn.naive_bayes import GaussianNB
clf_Gau = GaussianNB()
clf_Gau.fit(x_train,y_train)
y_pred_Gau = clf_Gau.predict(x_test)


In [23]:
# Multinominal Classifier
from sklearn.naive_bayes import MultinomialNB
clf_multi = MultinomialNB()
clf_multi.fit(x_train,y_train)
y_pred_multi = clf_multi.predict(x_test)

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

## comparing both algorithms

In [26]:
# sklearn in build  Gaussian classifier 
print(classification_report(y_test,y_pred_Gau))
print(confusion_matrix(y_test,y_pred_Gau))

                          precision    recall  f1-score   support

             alt.atheism       0.64      0.82      0.72       200
           comp.graphics       0.42      0.25      0.31       200
 comp.os.ms-windows.misc       0.55      0.27      0.36       200
comp.sys.ibm.pc.hardware       0.26      0.37      0.31       200
   comp.sys.mac.hardware       0.29      0.55      0.38       200
          comp.windows.x       0.58      0.19      0.29       200
            misc.forsale       0.42      0.78      0.55       200
               rec.autos       0.72      0.56      0.63       200
         rec.motorcycles       0.68      0.90      0.78       200
      rec.sport.baseball       0.69      0.82      0.75       200
        rec.sport.hockey       0.86      0.76      0.80       200
               sci.crypt       0.90      0.95      0.92       200
         sci.electronics       0.47      0.53      0.50       200
                 sci.med       0.79      0.38      0.51       200
         

In [27]:
# mulitnomial Classifier
print(classification_report(y_test,y_pred_multi))
print(confusion_matrix(y_test,y_pred_multi))

                          precision    recall  f1-score   support

             alt.atheism       0.78      0.62      0.69       200
           comp.graphics       0.65      0.76      0.70       200
 comp.os.ms-windows.misc       0.85      0.52      0.64       200
comp.sys.ibm.pc.hardware       0.63      0.79      0.70       200
   comp.sys.mac.hardware       0.78      0.90      0.83       200
          comp.windows.x       0.86      0.62      0.72       200
            misc.forsale       0.78      0.93      0.85       200
               rec.autos       0.88      0.84      0.86       200
         rec.motorcycles       0.77      0.93      0.84       200
      rec.sport.baseball       0.82      0.86      0.84       200
        rec.sport.hockey       0.92      0.71      0.81       200
               sci.crypt       0.95      0.81      0.88       200
         sci.electronics       0.67      0.80      0.73       200
                 sci.med       0.91      0.83      0.87       200
         

In [28]:
# self implementated classifier
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.87      0.69      0.77       200
           comp.graphics       0.95      0.89      0.92       200
 comp.os.ms-windows.misc       0.93      0.98      0.96       200
comp.sys.ibm.pc.hardware       0.99      0.99      0.99       200
   comp.sys.mac.hardware       0.99      0.97      0.98       200
          comp.windows.x       0.94      0.93      0.93       200
            misc.forsale       0.82      0.97      0.89       200
               rec.autos       0.99      0.98      0.99       200
         rec.motorcycles       0.98      0.98      0.98       200
      rec.sport.baseball       1.00      0.96      0.98       200
        rec.sport.hockey       0.98      0.99      0.99       200
               sci.crypt       0.96      0.83      0.89       200
         sci.electronics       0.87      0.96      0.91       200
                 sci.med       0.97      0.97      0.97       200
         

### self implemented classifier has the highest accuracy but is very slowly, followed by the  fast build-in multinomial classifier. The Gaussian Classifier is not very accurate.